# Import Dependencies

In [2]:
import pandas as pd
import numpy as np
import sklearn

# Read Dataframe

In [5]:
boning_df = pd.read_csv('Boning.csv')
slicing_df = pd.read_csv('Slicing.csv')

In [7]:
boning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54180 entries, 0 to 54179
Data columns (total 67 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Frame              54180 non-null  int64  
 1   L5 x               54180 non-null  float64
 2   L5 y               54180 non-null  float64
 3   L5 z               54180 non-null  float64
 4   L3 x               54180 non-null  float64
 5   L3 y               54180 non-null  float64
 6   L3 z               54180 non-null  float64
 7   T12 x              54180 non-null  float64
 8   T12 y              54180 non-null  float64
 9   T12 z              54180 non-null  float64
 10  T8 x               54180 non-null  float64
 11  T8 y               54180 non-null  float64
 12  T8 z               54180 non-null  float64
 13  Neck x             54180 non-null  float64
 14  Neck y             54180 non-null  float64
 15  Neck z             54180 non-null  float64
 16  Head x             541

In [8]:
slicing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 67 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Frame              17880 non-null  int64  
 1   L5 x               17880 non-null  float64
 2   L5 y               17880 non-null  float64
 3   L5 z               17880 non-null  float64
 4   L3 x               17880 non-null  float64
 5   L3 y               17880 non-null  float64
 6   L3 z               17880 non-null  float64
 7   T12 x              17880 non-null  float64
 8   T12 y              17880 non-null  float64
 9   T12 z              17880 non-null  float64
 10  T8 x               17880 non-null  float64
 11  T8 y               17880 non-null  float64
 12  T8 z               17880 non-null  float64
 13  Neck x             17880 non-null  float64
 14  Neck y             17880 non-null  float64
 15  Neck z             17880 non-null  float64
 16  Head x             178

# Data Processing

## Get only necessary columns

In [9]:
necessary_columns = ['Frame',
                     'Right Lower Leg x',
                     'Right Lower Leg y',
                     'Right Lower Leg z',
                     'Left Lower Leg x',
                     'Left Lower Leg y',
                     'Left Lower Leg z']

boning_df = boning_df[necessary_columns]
slicing_df = slicing_df[necessary_columns]

In [12]:
boning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54180 entries, 0 to 54179
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Frame              54180 non-null  int64  
 1   Right Lower Leg x  54180 non-null  float64
 2   Right Lower Leg y  54180 non-null  float64
 3   Right Lower Leg z  54180 non-null  float64
 4   Left Lower Leg x   54180 non-null  float64
 5   Left Lower Leg y   54180 non-null  float64
 6   Left Lower Leg z   54180 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 2.9 MB


In [11]:
slicing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Frame              17880 non-null  int64  
 1   Right Lower Leg x  17880 non-null  float64
 2   Right Lower Leg y  17880 non-null  float64
 3   Right Lower Leg z  17880 non-null  float64
 4   Left Lower Leg x   17880 non-null  float64
 5   Left Lower Leg y   17880 non-null  float64
 6   Left Lower Leg z   17880 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 977.9 KB


In [13]:
boning_df['label'] = 0
slicing_df['label'] = 1
dataset_df = pd.concat([boning_df, slicing_df], ignore_index=True)

In [14]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72060 entries, 0 to 72059
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Frame              72060 non-null  int64  
 1   Right Lower Leg x  72060 non-null  float64
 2   Right Lower Leg y  72060 non-null  float64
 3   Right Lower Leg z  72060 non-null  float64
 4   Left Lower Leg x   72060 non-null  float64
 5   Left Lower Leg y   72060 non-null  float64
 6   Left Lower Leg z   72060 non-null  float64
 7   label              72060 non-null  int64  
dtypes: float64(6), int64(2)
memory usage: 4.4 MB


## Composite Columns